# Orfeus Workshop 2022 (Potsdam)

This notebook contains simplified workflow for web services exposed by the EIDA (European Integrated Data Archive) Federation, focused on the EIDAWS-WFCatalog service and its functionalities allowing users to filter out low quality and low coverage data before downloading it to the local machine.

In this example we are going to:

1. Retrieve seismic event information from FDSNWS-Event catalogue offered by [GFZ](https://www.gfz-potsdam.de/) EIDA Node:
    * Date:
        * 📅 start date = 2020-01-01
        * 📅 end date = 2020-06-01
    * Event characteristics:
        * 🎚️ minimum magnitude = 5
    * Coordinates:
        * 🌐 minimum latitude = 40°N
        * 🌐 maximum latitude = 45°N
        * 🌐 minimum longitude = 17°E
        * 🌐 maximum longitude = 25°E
1. Using [FDSNWS-Station](https://www.orfeus-eu.org/data/eida/webservices/station/) web service, list all stations available in the region defined using epicentral distance.
1. Using [EIDAWS-WFCatalog](https://www.orfeus-eu.org/data/eida/webservices/wfcatalog/) service, we are going to exclude all stations which do not meet following criteria:
    * At least 95% data coverage on the day of the event
    * Maximum of 5 gaps
    * Sum of gaps lower than 50 seconds
    * No overlaps
1. Using [FDSNWS-Dataselect](https://www.orfeus-eu.org/data/eida/webservices/dataselect/) web service, we are going to download miniSEED files containing the waveforms from a time window defined using theoretial seismic wave arrival times.

## Imports

In [ ]:
import requests
import json
import pprint
from datetime import datetime, timedelta
from obspy import UTCDateTime
from obspy.geodetics import locations2degrees
from obspy.clients.fdsn.client import Client
from obspy.clients.fdsn import RoutingClient
from obspy.taup import TauPyModel

## Config

In [ ]:
# Define start and end dates for the event search
START = "2020-01-01"
END = "2020-06-01"

# Define event minimum magnitude and bounding box for event search
MAG_MIN = 5
LAT_MIN = 40
LAT_MAX = 45
LON_MIN = 17
LON_MAX = 25

# Max radius of stations from the epicenter (degrees)
RADIUS_MAX = 2

# Encoding for the HTTP requests
ENCODING = "utf-8"

# WFCatalog filter
AVAILABILITY_MIN = 95
GAPS_MAX = 5
GAPS_SUM_MAX = 50
OVERLAPS_MAX = 0

In [ ]:
# Global instances
CLIENT_GFZ = Client("GFZ")
CLIENT_EIDA = RoutingClient("eida-routing")

# We use the iasp91 reference model
TAUP_MODEL = TauPyModel(model="iasp91")

## Getting the event catalog

In [ ]:
catalog = CLIENT_GFZ.get_events(
    starttime=START,
    endtime=END,
    minmagnitude=MAG_MIN,
    minlatitude=LAT_MIN,
    maxlatitude=LAT_MAX,
    minlongitude=LON_MIN,
    maxlongitude=LON_MAX,
)
event = catalog[0]
print(event)

## Getting the station inventory

In [ ]:
inventory = CLIENT_EIDA.get_stations(
    startbefore=event.origins[0].time,
    endafter=event.origins[0].time,
    latitude=event.origins[0].latitude,
    longitude=event.origins[0].longitude,
    maxradius=RADIUS_MAX,
)
print(inventory)

## Validation function for the WFCatalog metrics

In [ ]:
def validate_station(string):
    """Our validating function which takes WFCatalog response json and returns
    list of network/station/channels for which all components are
    present and validated against our criteria.

    Args:
        string (string): JSON response from EIDAWS-WFCatalog web service

    Returns:
        []: List of validated networks, stations and channels
        None: If no channels have been found, function returns `None`
    """
    # Define mandatory channels
    required_channels = {
        "BH?": ["BHE", "BHN", "BHZ"],
        # "HH?": ["HHE", "HHN", "HHZ"],
        # "LH?": ["LHE", "LHN", "LHZ"],
    }

    # Parse string to json object
    j = json.loads(string)

    # Define channels_found as set to allow only unique values
    channels_found = set()
    channels_validated = []

    for cha in j:
        # Get network station channel identifiers
        network_code = cha["network"]
        station_code = cha["station"]
        channel_code = cha["channel"]

        # Get the quality metrics
        availability = int(cha["percent_availability"])
        gaps = int(cha["num_gaps"])
        sum_gaps = int(cha["sum_gaps"])
        overlaps = int(cha["num_overlaps"])

        msg = f"{network_code}.{station_code}.{channel_code}: {availability}% coverage, {gaps} gaps ({sum_gaps}s), {overlaps} overlaps."
        if (
            availability < AVAILABILITY_MIN
            or gaps > GAPS_MAX
            or sum_gaps > GAPS_SUM_MAX
            or overlaps > OVERLAPS_MAX
        ):
            print(f"NOK: {msg}")
        else:
            print(f"OK: {msg}")
            channels_found.add(channel_code.upper())

    # If channel components are present and validated, add them to the channels_validated list
    for c in required_channels.keys():
        if all(e in channels_found for e in required_channels[c]):
            channels_validated.append(c)

    if len(channels_validated) > 0:
        return ",".join(channels_validated)
    else:
        return None

## Getting list of stations filtered using WFCatalog metrics

In [ ]:
# List of verified stations, will be filled later
validated_stations = []

# Prepare from-to dates for the WFCatalog query
event_day = datetime(
    event.origins[0].time.year, event.origins[0].time.month, event.origins[0].time.day
)
event_day_next = event_day + timedelta(days=1)

for net in inventory.networks:
    for sta in net.stations:
        wfcatalog_url = "http://eida-federator.ethz.ch/eidaws/wfcatalog/1/query"

        query_params = {
            "start": event_day.isoformat(),
            "end": event_day_next.isoformat(),
            "network": net.code,
            "station": sta.code,
        }

        # Request the data...
        r = requests.get(wfcatalog_url, params=query_params, timeout=10)
        r.encoding = ENCODING

        if r.status_code == 200:
            # Validate station
            cha = validate_station(r.text)
            if cha:
                # Validation passed, add to verified_stations list
                validated_stations.append(
                    {
                        "network": net.code,
                        "station": sta.code,
                        "latitude": sta.latitude,
                        "longitude": sta.longitude,
                        "channels": cha,
                    }
                )
        else:
            pass

# Let's print our validated stations
pp = pprint.PrettyPrinter()
pp.pprint(validated_stations)

## Function calculating theoretical seismic wave arrival time at station

In [ ]:
def getPArrival(event, station):
    # Determine the arc distance using the haversine formula
    arcDistanceDegrees = locations2degrees(
        event.origins[0].latitude,
        station["latitude"],
        event.origins[0].longitude,
        station["longitude"],
    )

    # Calculate the theoretical P-arrival time
    arrivals = TAUP_MODEL.get_travel_times(
        source_depth_in_km=1e-3 * event.origins[0].depth,
        distance_in_degree=arcDistanceDegrees,
        phase_list=["P"],
    )

    # Add the theorical P-arrival delta to the event time
    return UTCDateTime(event.origins[0].time) + arrivals[0].time

## Downloading and plotting the waveforms

In [ ]:
for station in validated_stations:
    stationArrivalTime = getPArrival(event, station)
    st = CLIENT_EIDA.get_waveforms(
        network=station["network"],
        station=station["station"],
        channel=station["channels"],
        starttime=(stationArrivalTime - 600).isoformat(),
        endtime=(stationArrivalTime + 1200).isoformat(),
    )
    print(st)
    st.plot()
    # Break after first station for better readibility
    break